In [2]:
from katabatic.models.TableGAN import TableGANAdapter, TableGAN, preprocess_data, postprocess_data
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

train_data_path = 'data/PokerHand/poker-hand-training-true.data'
test_data_path = 'data/PokerHand/poker-hand-testing.data'

# Initialize the adapter with a specific privacy setting
tablegan_adapter = TableGANAdapter(type='continuous', privacy_setting='high')
# load data
df_train = pd.read_csv(train_data_path, header = None)
df_test = pd.read_csv(test_data_path, header = None)
x_train = df_train.drop(10, axis = 1).values
y_train = df_train[10].values
x_test = df_test.drop(10, axis = 1).values
y_test = df_test[10].values

# Fit the model
#tablegan_adapter.fit(x_train, y_train, epochs=200, batch_size=64)
tablegan_adapter.fit(x_train, y_train, epochs=100, batch_size=64)
#tablegan_adapter.fit(x_train, y_train, epochs=25, batch_size=64)

---FIT TableGAN Model with high privacy setting
---Initialise TableGAN Model


d:\Anaconda3\envs\py39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\Anaconda3\envs\py39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 10/100: [D loss: -0.5744] [G loss: 1.1233] [C loss: 0.9404]
Epoch 20/100: [D loss: -0.3417] [G loss: 1.5270] [C loss: 0.9075]
Epoch 30/100: [D loss: -0.3101] [G loss: 2.0770] [C loss: 0.8507]
Epoch 40/100: [D loss: -0.3058] [G loss: 2.0176] [C loss: 0.7879]
Epoch 50/100: [D loss: -0.3303] [G loss: 3.3233] [C loss: 0.7254]
Epoch 60/100: [D loss: -0.3206] [G loss: 3.2439] [C loss: 0.6727]
Epoch 70/100: [D loss: -0.3863] [G loss: 2.8209] [C loss: 0.6295]
Epoch 80/100: [D loss: -0.4183] [G loss: 0.0558] [C loss: 0.5917]
Epoch 90/100: [D loss: -0.3886] [G loss: -1.6258] [C loss: 0.5598]
Epoch 100/100: [D loss: -0.3947] [G loss: -1.0898] [C loss: 0.5300]


In [5]:
# Generate synthetic data
synthetic_data = tablegan_adapter.generate(size=25010)
#synthetic_data = tablegan_adapter.generate(size=100)

# Check the shape of synthetic data
synthetic_df = pd.DataFrame(synthetic_data).astype(int)
x_sync_train = synthetic_df.drop(10, axis = 1).values
y_sync_train = synthetic_df[10].values

---Generate from TableGAN Model


In [6]:
import numpy as np
# evaluate
ohe = OneHotEncoder(handle_unknown='ignore')
lbe = LabelEncoder()
#y_train = np.append(y_train, -1)
#y_sync_train = np.append(y_sync_train, -1)
for index, value in enumerate(y_test):
    if value not in y_sync_train:
        y_test[index] = 0

# TRTR
x_train_ohe = ohe.fit_transform(x_train)
x_test_ohe = ohe.transform(x_test)
y_train_lbe = lbe.fit_transform(y_train).astype(int)
y_test_lbe = lbe.transform(y_test).astype(int)
trtr_score_lr  = LogisticRegression().fit(x_train_ohe, y_train_lbe).score(x_test_ohe, y_test_lbe)
trtr_score_rf  = RandomForestClassifier().fit(x_train, y_train_lbe).score(x_test, y_test_lbe)
trtr_score_mlp = MLPClassifier().fit(x_train_ohe, y_train_lbe).score(x_test_ohe, y_test_lbe)
xgbt_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
trtr_score_xgbt = xgbt_classifier.fit(x_train_ohe, y_train_lbe).score(x_test_ohe, y_test_lbe)

# TSTR
x_train_ohe = ohe.fit_transform(x_sync_train)
x_test_ohe = ohe.transform(x_test)
y_train_lbe = lbe.fit_transform(y_sync_train).astype(int)
y_test_lbe = lbe.transform(y_test).astype(int)
tstr_score_lr  = LogisticRegression().fit(x_train_ohe, y_train_lbe).score(x_test_ohe, y_test_lbe)
tstr_score_rf  = RandomForestClassifier().fit(x_train, y_train_lbe).score(x_test, y_test_lbe)
tstr_score_mlp = MLPClassifier().fit(x_train_ohe, y_train_lbe).score(x_test_ohe, y_test_lbe)
xgbt_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
tstr_score_xgbt = xgbt_classifier.fit(x_train_ohe, y_train_lbe).score(x_test_ohe, y_test_lbe)

df_evaluate = pd.DataFrame([
    ['TSTR', tstr_score_lr, tstr_score_rf, tstr_score_mlp, tstr_score_xgbt],
    ['TRTR', trtr_score_lr,trtr_score_rf,trtr_score_mlp, trtr_score_xgbt]
], columns=['Evaluated Item', 'LR', 'RF', 'MLP', 'XGBT'])
print(df_evaluate)

d:\Anaconda3\envs\py39\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Anaconda3\envs\py39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\Anaconda3\envs\py39\lib\site-packages\xgboost\core.py:158: UserWarning: [13:47:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_en

  Evaluated Item        LR        RF       MLP      XGBT
0           TSTR  0.472759  0.492685  0.636337  0.666819
1           TRTR  0.501067  0.616169  0.977174  0.931637
